# Step 3: Validation

In [1]:
import numpy as np
import pandas as pd
import librosa
import os
import matplotlib.pyplot as plt
%matplotlib inline
from PIL import Image as im
import tensorflow as tf

In [2]:
target = ['redcro', 'comrav', 'houspa', 'norcar', 'barswa', 'houwre', 'sonspa',
       'gbwwre1', 'eursta', 'spotow', 'cubthr', 'rewbla', 'mallar3', 'amerob',
       'bewwre', 'carwre', 'swathr', 'rubpep1', 'bncfly', 'rucspa1']

In [42]:
meta = pd.read_csv("../birdsdata/train_soundscape_labels.csv")
meta.head()
target_file = []
for line in meta.iterrows():
    bird_types = line[1][4]
    file = line[1][0].split("_")
    file = file[0] + "_" + file[1]
    bird_types = bird_types.split(" ")
    for bird in bird_types:
        if bird in target:
            if file not in target_file:
                target_file.append(file)
print(target_file)



['10534_SSW', '26709_SSW', '28933_SSW', '51010_SSW']


## load models

In [5]:
mobile_model = tf.keras.models.load_model("mobile_v2/mobile_model")

2022-05-15 15:53:38.432503: I tensorflow/core/platform/cpu_feature_guard.cc:145] This TensorFlow binary is optimized with Intel(R) MKL-DNN to use the following CPU instructions in performance critical operations:  SSE4.1 SSE4.2
To enable them in non-MKL-DNN operations, rebuild TensorFlow with the appropriate compiler flags.
2022-05-15 15:53:38.434149: I tensorflow/core/common_runtime/process_util.cc:115] Creating new thread pool with default inter op setting: 8. Tune using inter_op_parallelism_threads for best performance.


## validation with Real long calls file

In [6]:
FMIN = 500
FMAX = 12500
SPEC_HEIGHT = 64
SPEC_WIDTH = 256
NUM_MELS = SPEC_HEIGHT
HOP_LENGTH = int(32000 * 5 / (SPEC_WIDTH - 1))

In [51]:
sound_path = "../birdsdata/train_soundscapes/"
result = {"raw_id":[], "prediction":[], "score":[]}

for record in os.listdir(sound_path):
    file_as_list = record.split("_")
    file = file_as_list[0] + "_" + file_as_list[1]
    if file in target_file:
        signal, reatio = librosa.load(sound_path+record, sr=32000)
        frag_len = 480000
        signals = []
        for i in range(0, len(signal), frag_len):
            sig_frag = signal[i:i+frag_len]
            if len(sig_frag) < frag_len:
                break
            signals.append(sig_frag)
        count = 0
        for frag in signals:
            count += 1
            spec = librosa.feature.melspectrogram(y=frag, 
                                                sr=32000, 
                                                n_fft=1024,
                                                hop_length=HOP_LENGTH, 
                                                n_mels=NUM_MELS,
                                                fmin=FMIN,
                                                fmax=FMAX)
            spec = librosa.power_to_db(spec, ref=np.max)
            spec -= spec.min()
            spec /= spec.max()
            spec = np.expand_dims(spec, -1)
            spec = np.expand_dims(spec, 0)
            predict = mobile_model.predict(spec)
            idx = predict[0].argmax()
            prediction = target[idx]
            score = predict[0][idx]
            if score < 0.5:
                prediction = "nocall"
                score = 1 - score
            result["raw_id"].append(file+"_"+str(5*count))
            result["prediction"].append(prediction)
            result["score"].append(score)
            
            
result_pd = pd.DataFrame(result, columns=["raw_id", "prediction", "score"])
result_pd.head(30)
result_pd.to_csv("result.csv")

In [67]:
sound_path = "../birdsdata/test_data/"
result = {"raw_id":[], "prediction":[], "score":[]}

for record in os.listdir(sound_path):
    if record.startswith("."):
        continue
    file_as_list = record.split(".")
    file = file_as_list[0]
    signal, reatio = librosa.load(sound_path+record, sr=32000)
    frag_len = 480000
    signals = []
    for i in range(0, len(signal), frag_len):
        sig_frag = signal[i:i+frag_len]
        if len(sig_frag) < frag_len:
            break
        signals.append(sig_frag)
    count = 0
    confidence = {}
    for frag in signals:
        count += 1
        spec = librosa.feature.melspectrogram(y=frag, 
                                            sr=32000, 
                                            n_fft=1024,
                                            hop_length=HOP_LENGTH, 
                                            n_mels=NUM_MELS,
                                            fmin=FMIN,
                                            fmax=FMAX)
        spec = librosa.power_to_db(spec, ref=np.max)
        spec -= spec.min()
        spec /= spec.max()
        spec = np.expand_dims(spec, -1)
        spec = np.expand_dims(spec, 0)
        predict = mobile_model.predict(spec)
        idx = predict[0].argmax()
        prediction = target[idx]
        score = predict[0][idx]
        if score < 0.48:
            prediction = "nocall"
            score = 1 - score
        result["raw_id"].append(file+"_"+str(15*count))
        result["prediction"].append(prediction)
        result["score"].append(score)
        if prediction in confidence.keys():
            confidence[prediction] += score
        else:
            confidence[prediction] = score
    highest_conf = 0
    most_conf = "nocall"
    for predicts, conf in confidence.items():
        if conf > highest_conf:
            most_conf = predicts
            highest_conf = conf
    print("for the recording file: " + file + " is the type: " + most_conf)
result_pd = pd.DataFrame(result, columns=["raw_id", "prediction", "score"])
result_pd.head(30)
result_pd.to_csv("result_online.csv")

for the recording file: XC702290-家麻雀-Passer-domesticus is the type: houspa
for the recording file: AnyConv is the type: barswa
for the recording file: AnyConv is the type: rewbla
for the recording file: XC720943 - 褐冠蝇霸鹟 - Myiarchus tyrannulus is the type: amerob
for the recording file: XC720636-家麻雀-Passer-domesticus is the type: houspa
for the recording file: XC718664-家麻雀-Passer-domesticus is the type: bncfly
for the recording file: XC722456-家麻雀-Passer-domesticus is the type: houspa
for the recording file: AnyConv is the type: nocall
for the recording file: AnyConv is the type: rewbla
for the recording file: AnyConv is the type: rewbla
for the recording file: XC704329 - 褐冠蝇霸鹟 - Myiarchus tyrannulus is the type: bncfly
for the recording file: XC722397-家麻雀-Passer-domesticus is the type: houspa
